<a href="https://colab.research.google.com/github/afullhart/climateanalogs/blob/main/Colab/Score.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%reset -f

In [5]:
!pip install rioxarray
!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.8 MB/s eta 0:00:00


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd
from sklearn.metrics import silhouette_samples, silhouette_score
import rasterio
import rioxarray as rx

src = rasterio.open("your_geotiff.tif")

raster_data = rx.open_rasterio("your_geotiff.tif")
